#Necessary imports

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import warnings
warnings.filterwarnings("ignore")

In [29]:
!pip install -q lightly

In [30]:
!pip install wandb -Uq

In [31]:
!pip install -U openmim && mim install "mmpretrain>=1.0.0rc8"

Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.12.0/index.html


In [32]:
N_EPOCHS = 20

In [33]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [34]:
import numpy as np
import pandas as pd
import copy
import torch
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from torch import nn
import matplotlib.pyplot as plt
from PIL import Image

from tqdm import tqdm

import torch
import torchvision

from torch import nn
from torchvision import transforms

from lightly.loss import NTXentLoss
from lightly.models.modules import SimCLRProjectionHead
from lightly.transforms.simclr_transform import SimCLRTransform

from lightly.data import LightlyDataset
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda', index=0)

In [35]:
from knn import KNN, reproducibility
reproducibility(1000)

#Define model

In [36]:
from mmpretrain import get_model

model = get_model('vit-small-p14_dinov2-pre_3rdparty', pretrained=False)
model = model.to(DEVICE)

In [37]:
# model.load_state_dict(torch.load('dino_TIN_acc10.00_epoch30.pt'))

#Data (cifar10 or cifar100)

In [38]:
from lightly.transforms.dino_transform import DINOTransform

In [39]:
from torchvision import datasets
import os

transform = SimCLRTransform(input_size=32, gaussian_blur=0.0)
dst_train = datasets.ImageFolder(root='datasets/tiny-imagenet-200/train', transform=transform)

# data_mean, data_std = get_input_stats(DATASET)
# normalize_dict = {'mean': data_mean, 'std': data_std}

dataset = LightlyDataset.from_torch_dataset(dst_train, transform=transform)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=128,
    shuffle=True,
    drop_last=True,
    num_workers=2,
    pin_memory=True
)

#Optimizer and Scheduler

In [40]:
criterion = NTXentLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.999)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,base_lr=0.001, max_lr=0.01)

#KNN and dataloaders for evaluation

In [41]:
from torchvision import transforms as T
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

BATCH_SIZE = 32

test_transform = T.Compose([
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ])

def get_loaders( batch_size, transform=test_transform,data_path='./'):
    dst_train = datasets.ImageFolder(root=os.path.join(data_path,'datasets/tiny-imagenet-200/train'), transform=test_transform)
    dst_test = datasets.ImageFolder(root=os.path.join(data_path, 'datasets/tiny-imagenet-200/val'), transform=test_transform)
    train_loader = DataLoader(dataset=dst_train, batch_size=batch_size, num_workers=2, drop_last=True)
    test_loader = DataLoader(dataset=dst_test, batch_size=batch_size, num_workers=2, drop_last=True)
    return train_loader, test_loader

train_loader, test_loader = get_loaders(batch_size=BATCH_SIZE)

In [42]:
# import time
# model.eval()
# start_time = time.time()
# ssl_evaluator = KNN(model=model, k=1, device='cuda', transformer=True)
# train_acc, val_acc = ssl_evaluator.fit(train_loader, test_loader)
# print(train_acc, val_acc)
# print(time.time() - start_time, 'seconds')

#Make directory for saving models & Training

In [43]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [44]:
config = dict(
    epochs=N_EPOCHS,
    classes=200,
    learning_rate=3e-4,
    dataset="TinyImageNet",
    model="BarlowTwins")

In [45]:
wandb.init(project="Dino_TIN_20(early_finish)epochs", config=config)

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██▁▁▂▂
loss,▇▆▆▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁██▇▇
epoch,3
loss,4.45504


In [46]:
best_val_acc = 0

print("Starting Training")
for epoch in tqdm(range(N_EPOCHS)):
    total_loss = 0
    for batch in dataloader:
        x0, x1 = batch[0]
        x0 = x0.to(device)
        x1 = x1.to(device)
        z0 = model.extract_feat(x0)[0]
        z1 = model.extract_feat(x1)[0]
        loss = criterion(z0, z1)
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    # scheduler.step()
    avg_loss = total_loss / len(dataloader)
    print(f" epoch: {epoch:>02}, loss: {avg_loss:.5f}")
    wandb.log({'epoch': epoch, 'loss': avg_loss})
    # if epoch % 10 == 0:
    #     ssl_evaluator = KNN(model=model, k=1, device='cuda')
    #     train_acc, val_acc = ssl_evaluator.fit(loader_train_plain, loader_test)
    #     if val_acc > best_val_acc:
    #         torch.save(model.state_dict(),f'dino_{DATASET}_acc{val_acc:.2f}.pt')
    #         best_val_acc = val_acc
    #     print(f' Train Accuracy:{train_acc:.1f}%',f' Val Accuracy:{val_acc:.1f}%')

Starting Training


  5%|▌         | 1/20 [05:33<1:45:32, 333.27s/it]

 epoch: 00, loss: 5.05278


 10%|█         | 2/20 [11:03<1:39:30, 331.70s/it]

 epoch: 01, loss: 4.83692


...

 epoch: 17, loss: 4.54815


 95%|█████████▌| 19/20 [1:46:21<05:36, 336.30s/it]

 epoch: 18, loss: 4.53579


100%|██████████| 20/20 [1:52:03<00:00, 336.16s/it]

 epoch: 19, loss: 4.53066


#Load best model we have got

In [47]:
print('!')
ssl_evaluator = KNN(model=model, k=1, device='cuda', transformer=True)
train_acc, val_acc = ssl_evaluator.fit(train_loader, test_loader)
print(train_acc, val_acc)

!
Evaluate on train data...
Evaluate on test data...
4.1579999999999995 3.5456340823861043


#Visual evaluation

#Save backbone model

In [48]:
backbone = model.backbone
torch.save(backbone.state_dict(), f'dino_TIN_acc{val_acc:.2f}_epoch20.pt')